In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU Available: True
GPU Name: Tesla T4


In [ ]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas

od.download("https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: reemramadannegm
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset


100%|██████████| 149M/149M [00:00<00:00, 1.23GB/s]

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import cv2
import time
import random
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tempfile import TemporaryDirectory
from sklearn.metrics import confusion_matrix, accuracy_score
from torchvision import models, transforms, datasets
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from torch.optim import lr_scheduler

In [ ]:
data_path = '/content/brain-tumor-mri-dataset'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
batch_size = 16
learning_rate = 0.001
num_epochs = 10

Using cuda device


In [ ]:
data_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
])

In [ ]:
from torchvision import models

def tranfer_learning(model_name):
    if model_name == 'efficientnet_b2':
        model = models.efficientnet_b2(pretrained=True)
        num_ft = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ft, 4)
        model = model.to(device)
        return model



In [ ]:
def train_and_validate(model, criterion, optimizer, scheduler, num_epochs=15):
    since = time.time()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['Training', 'Testing']:
            if phase == 'Training':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            with torch.set_grad_enabled(phase == 'Training'):
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'Training':
                        loss.backward()
                        optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            if phase == 'Training':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'Testing' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), 'full_model.pth')

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Testing Acc: {best_acc:.4f}')

    model.load_state_dict(torch.load('full_model.pth'))
    return model


In [ ]:
from sklearn.metrics import classification_report

def final_test_report(model):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloaders['Testing']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Final Test Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names, zero_division=0))


In [ ]:
model = tranfer_learning('efficientnet_b2')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, gamma=0.1)
model = train_and_validate(model, criterion, optimizer, exp_lr_scheduler, num_epochs=5)
final_test_report(model)


Epoch 1/5
----------
Training Loss: 0.3333 Acc: 0.8871
Testing Loss: 0.2310 Acc: 0.9146

Epoch 2/5
----------
Training Loss: 0.1610 Acc: 0.9480
Testing Loss: 0.1117 Acc: 0.9588

Epoch 3/5
----------
Training Loss: 0.0979 Acc: 0.9697
Testing Loss: 0.0548 Acc: 0.9840

Epoch 4/5
----------
Training Loss: 0.0884 Acc: 0.9720
Testing Loss: 0.8782 Acc: 0.8528

Epoch 5/5
----------
Training Loss: 0.0824 Acc: 0.9744
Testing Loss: 0.0732 Acc: 0.9779

Training complete in 5m 59s
Best Testing Acc: 0.9840
Final Test Classification Report:
              precision    recall  f1-score   support

      glioma       0.95      0.99      0.97       300
  meningioma       0.98      0.96      0.97       306
     notumor       1.00      0.98      0.99       405
   pituitary       0.99      0.99      0.99       300

    accuracy                           0.98      1311
   macro avg       0.98      0.98      0.98      1311
weighted avg       0.98      0.98      0.98      1311



In [ ]:
torch.save(model, 'full_model.pth')
